#### Q8 : Write a Python Program to implement FP tree growth Algorithm on a dataset of your own choice.

#### Ans:

#### FP Tree growth Algorithm

In [2]:
from collections import defaultdict, Counter

In [4]:
class FPNode:
    """
    A node in the FP-tree.
    """
    def __init__(self, item, count, parent):
        self.item = item          # item name
        self.count = count        # frequency count
        self.parent = parent      # parent FPNode
        self.children = {}        # item -> FPNode
        self.node_link = None     # next node with same item (for header table)

    def increment(self, count):
        """
        Increment the count of this node.
        """
        self.count += count



In [5]:

class FPTree:
    """
    FP-tree structure.
    """
    def __init__(self, transactions, min_support):
        self.min_support = min_support
        self.root = FPNode(None, 1, None)  # root is a special null node
        self.header_table = {}  # item -> [support, node_link]

        # Step 1: count item frequencies
        item_counts = Counter()
        for transaction in transactions:
            item_counts.update(transaction)

        # Step 2: remove items that don't meet min_support
        self.freq_items = {item: cnt for item, cnt in item_counts.items()
                           if cnt >= self.min_support}

        # If no items are frequent, tree is empty
        if not self.freq_items:
            return

        # Step 3: build header table: item -> [support, first_node_link(None)]
        for item, cnt in self.freq_items.items():
            self.header_table[item] = [cnt, None]

        # Step 4: build the FP-tree
        for transaction in transactions:
            # Filter infrequent items and sort by global frequency (descending)
            filtered_trans = [item for item in transaction if item in self.freq_items]
            filtered_trans.sort(key=lambda item: self.freq_items[item], reverse=True)

            # Insert into tree
            self._insert_tree(filtered_trans, self.root)

    def _insert_tree(self, items, node):
        """
        Recursively insert items into the FP-tree.
        """
        if not items:
            return

        first_item = items[0]
        # If child exists, increment; otherwise create new node
        if first_item in node.children:
            child = node.children[first_item]
            child.increment(1)
        else:
            child = FPNode(first_item, 1, node)
            node.children[first_item] = child

            # Update header table to link this node
            self._update_header_table(first_item, child)

        # Recursively insert the remaining items
        remaining_items = items[1:]
        self._insert_tree(remaining_items, child)

    def _update_header_table(self, item, new_node):
        """
        Add new_node to the linked list of nodes for this item (header table).
        """
        _, first_node = self.header_table[item]
        if first_node is None:
            self.header_table[item][1] = new_node
        else:
            # Traverse node_link chain and append at the end
            current = first_node
            while current.node_link is not None:
                current = current.node_link
            current.node_link = new_node

    def is_empty(self):
        """
        Check if FP-tree has no frequent items.
        """
        return len(self.header_table) == 0


def ascend_tree(node):
    """
    Ascend from node to root, collecting the path (excluding root and the node itself).
    """
    path = []
    while node.parent is not None and node.parent.item is not None:
        node = node.parent
        path.append(node.item)
    return path


def find_conditional_pattern_base(item, header_table):
    """
    For an item, find its conditional pattern base.
    Returns a list of (path, count).
    """
    conditional_patterns = []
    _, node = header_table[item]

    # Traverse linked list of nodes for this item
    while node is not None:
        path = ascend_tree(node)
        if path:
            conditional_patterns.append((path, node.count))
        node = node.node_link
    return conditional_patterns


def construct_conditional_tree(conditional_patterns, min_support):
    """
    Construct a conditional FP-tree from conditional pattern base.
    conditional_patterns: list of (path, count)
    """
    # Rebuild transactions from patterns with counts
    transactions = []
    for path, count in conditional_patterns:
        for _ in range(count):
            transactions.append(path)

    # Build new FP-tree
    return FPTree(transactions, min_support)


def mine_tree(tree, prefix, min_support, frequent_itemsets):
    """
    Recursively mine the FP-tree to find all frequent itemsets.
    prefix: current prefix (list of items)
    frequent_itemsets: dict mapping frozenset -> support
    """
    # Sort items in header table by frequency (ascending for FP-growth)
    items = sorted(tree.header_table.items(), key=lambda x: x[1][0])

    for item, (support, _) in items:
        new_freq_set = prefix.copy()
        new_freq_set.append(item)
        frequent_itemsets[frozenset(new_freq_set)] = support

        # Build conditional pattern base for this item
        conditional_pattern_base = find_conditional_pattern_base(item, tree.header_table)

        # Build conditional FP-tree
        conditional_tree = construct_conditional_tree(conditional_pattern_base, min_support)

        # If the conditional tree is not empty, recurse
        if not conditional_tree.is_empty():
            mine_tree(conditional_tree, new_freq_set, min_support, frequent_itemsets)


#### Example

In [6]:
if __name__ == "__main__":
    # Sample dataset (transactions)
    transactions = [
        ['bread', 'milk'],
        ['bread', 'diaper', 'beer', 'egg'],
        ['milk', 'diaper', 'beer', 'cola'],
        ['bread', 'milk', 'diaper', 'beer'],
        ['bread', 'milk', 'diaper', 'cola'],
    ]

    # Minimum support threshold
    min_support = 2

    # Build the FP-tree
    fp_tree = FPTree(transactions, min_support)

    # Mine frequent itemsets
    frequent_itemsets = {}
    if not fp_tree.is_empty():
        mine_tree(fp_tree, prefix=[], min_support=min_support, frequent_itemsets=frequent_itemsets)

    # Print results
    print("Frequent Itemsets (min_support = {}):".format(min_support))
    for itemset, support in sorted(frequent_itemsets.items(), key=lambda x: (len(x[0]), x[0])):
        print(f"{set(itemset)} : {support}")


Frequent Itemsets (min_support = 2):
{'cola'} : 2
{'beer'} : 3
{'bread'} : 4
{'milk'} : 4
{'diaper'} : 4
{'cola', 'diaper'} : 2
{'cola', 'milk'} : 2
{'beer', 'bread'} : 2
{'milk', 'beer'} : 2
{'diaper', 'beer'} : 3
{'milk', 'bread'} : 3
{'diaper', 'bread'} : 3
{'diaper', 'milk'} : 3
{'cola', 'diaper', 'milk'} : 2
{'diaper', 'beer', 'bread'} : 2
{'milk', 'diaper', 'beer'} : 2
{'diaper', 'milk', 'bread'} : 2
